In [1]:
%load_ext watermark
%watermark

Last updated: 2024-05-25T12:01:00.171897-05:00

Python implementation: CPython
Python version       : 3.10.11
IPython version      : 8.13.2

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing, feature_extraction
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
#import category_encoders
from sklearn.model_selection import cross_validate
from category_encoders import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.compose import ColumnTransformer

In [3]:
consumidores = pd.read_csv("C:/Users/Danny/Desktop/Analisis tesis/4 - Pipelines/data/df_consumo_varObjetivo.csv")

In [4]:
consumidores.head()

,departamento,residentes_hogar,d2_04_num_hijos,per_edad_tipo,estrato_tipo,situacion_tipo,frecuencia_consumo_marihuana_tipo,frecuencia_consumo_cocaina_tipo,frecuencia_consumo_basuco_tipo,per_sexo_tipo,...,g_06_a_posibilidad_conseguir_marihuana_tipo,g_06_b_posibilidad_conseguir_cocaina_tipo,g_06_c_posibilidad_conseguir_basuco_tipo,g_07_alguien_ofrecio_comprar_probar_sp_tipo,g_01_a_num_familiares_consumen_sp_imp_tipo,g_02_a_num_amigos_consumen_sp_imp_tipo,g_08_a_ofrecieron_marihuana_imp_tipo,g_08_b_ofrecieron_cocaina_imp_tipo,g_08_c_ofrecieron_basuco_imp_tipo,CatRiesgo
0,Guaviare,5,0,Joven,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
1,Cordoba,4,3,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
2,Caqueta,4,2,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
3,Bolivar,6,0,Adulto,Estrato alto,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_No_podria_conseguir,g_06_b_No_podria_conseguir,g_06_c_No_podria_conseguir,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
4,Tolima,4,1,Joven,Estrato bajo,Buscando trabajo,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,1


En este modelo vamos a modificar cada variable en función de su tipo. Al conjunto de pasos que siguen los datos se le llama comúnmente **Pipelines** (literalmente, sistemas de tuberias).

In [5]:
consumidores.shape

(10000, 40)

**INCISO**

Es necesario modificar dos transformadores de scikitlearn para que sean compatibles con pipelines. Este paso es necesario en la version actual de scikit-learn

In [6]:
variable_objetivo = ["CatRiesgo"]
variables_independientes = consumidores.drop(variable_objetivo, axis=1).columns

In [7]:
datos_numericos = consumidores[variables_independientes].select_dtypes([int, float])
col_numericas = datos_numericos.columns
col_no_numericas = consumidores[variables_independientes].select_dtypes([object]).columns

dict_var_ordinales = {
    "per_edad_tipo": ["Adolescente", "Joven", "Adulto", "Persona mayor"],
    "estrato_tipo":["Estrato bajo", "Estrato medio", "Estrato alto"],
    "d_08_estado_salud_tipo":["d_08_Muy_malo","d_08_Malo","d_08_Regular","d_08_Bueno","d_08_Muy_bueno"],
    "frecuencia_consumo_marihuana_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_cocaina_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_basuco_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
}

col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = consumidores[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = consumidores[col_categoricas]

Un pipeline de sklearn se define como una secuencia de pasos. Cada paso se define con una tupla de forma `[nombre del paso, transformador]`

Por ejemplo, si queremos crear un pipeline que procese las variables numéricas, primero imputándolas y después estandarizandolas, podriamos crear un pipeline

Creamos ahora los pipelines para cada tipo de variable. Es necesario aclarar que se va ha hacer uso de la libreria ColumnTransformer para el procesamiento de variables categoricas y categoricas ordinales

In [8]:
transformador_numerico = Pipeline([
    ("imputador_numerico", SimpleImputer(missing_values=np.nan, strategy='mean')),
    ("escalador_numerico", preprocessing.MinMaxScaler())
])

pipeline_numerico = ColumnTransformer([
    ("transformador_numerico", transformador_numerico, col_numericas),
])

pipeline_numerico.fit_transform(consumidores)[:5]

array([[0.30769231, 0.        ],
       [0.23076923, 0.25      ],
       [0.23076923, 0.16666667],
       [0.38461538, 0.        ],
       [0.23076923, 0.08333333]])

In [9]:
pipeline_numerico

ColumnTransformer(transformers=[('transformador_numerico',
                                 Pipeline(steps=[('imputador_numerico',
                                                  SimpleImputer()),
                                                 ('escalador_numerico',
                                                  MinMaxScaler())]),
                                 Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])

Ahora se procede a crear el pipeline categorico

In [10]:
transformador_categorico = ColumnTransformer([
    ("transformador_categorico", OneHotEncoder(handle_unknown="impute"), col_categoricas)
])

pipeline_categorico = Pipeline([
    ("transformador_categorico", transformador_categorico)
])

pipeline_categorico.fit_transform(consumidores)[:5]

array([[1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,

In [11]:
pipeline_categorico

Pipeline(steps=[('transformador_categorico',
                 ColumnTransformer(transformers=[('transformador_categorico',
                                                  OneHotEncoder(handle_unknown='impute'),
                                                  ['g_06_b_posibilidad_conseguir_cocaina_tipo',
                                                   'per_sexo_tipo',
                                                   'g_02_amigos_consumen_sp_tipo',
                                                   'g_03_curiosidad_probar_sp_tipo',
                                                   'g_05_posibilidad_probar_sp_tipo',
                                                   'd_09_deprimido_tipo',
                                                   'd_10_poco_interes_tipo',
                                                   '...
                                                   'g_08_c_ofrecieron_basuco_imp_tipo',
                                                   'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                   'd_12_c_presenta_problema_expendio_sp_barrio_tipo',
                                                   'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                   'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                   'd_01_aporta_dinero_hogar_tipo',
                                                   'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                   'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo', ...])]))])

`ColumnTransformer` requiere un mapeado específico para cada columna, indicando el número por el que queremos reemplazar cada nivel de cada variable ordinal. Si no especificamos un mapa, asume que el orden es aleatorio.

In [12]:
niveles_ordinales = []
for col, levels in dict_var_ordinales.items():
    niveles_ordinales.append({"col": col, 
                              "mapping": dict(zip(levels, range(len(levels))))})

In [13]:
transformador_ordinal = ColumnTransformer([
    ("transformador_ordinal", OrdinalEncoder(mapping=niveles_ordinales), col_ordinales)
])

pipeline_ordinal = Pipeline([
    ("transformador_ordinal", transformador_ordinal)
])
pipeline_ordinal.fit_transform(consumidores)[:5]

array([[1, 0, 1, 0, 0, 0],
       [2, 0, 2, 0, 0, 0],
       [2, 0, 3, 0, 0, 0],
       [2, 2, 3, 0, 0, 0],
       [1, 0, 3, 0, 0, 0]])

Ya tenemos una manera de, dado un conjunto de datos, separarlos y aplicar distintas transformaciones a cada variable. Nos falta una manera de, una vez se han transformado, reunirlas de nuevo.

Para ello podemos usar `FeatureUnion`, que simplemente toma un conjunto de pasos de un pipeline y los une.

In [14]:
pipeline_procesado = FeatureUnion([
    ('variables_numericas', pipeline_numerico),
    ('variables_ordinales', pipeline_ordinal),
    ('variables_categoricas', pipeline_categorico),
])

In [15]:
pipeline_procesado

FeatureUnion(transformer_list=[('variables_numericas',
                                ColumnTransformer(transformers=[('transformador_numerico',
                                                                 Pipeline(steps=[('imputador_numerico',
                                                                                  SimpleImputer()),
                                                                                 ('escalador_numerico',
                                                                                  MinMaxScaler())]),
                                                                 Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                               ('variables_ordinales',
                                Pipeline(steps=[('transformador_ordinal',
                                                 ColumnTransfor...
                                                                                   'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                   'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                   'd_12_c_presenta_problema_expendio_sp_barrio_tipo',
                                                                                   'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                   'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                   'd_01_aporta_dinero_hogar_tipo',
                                                                                   'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                   'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo', ...])]))]))])

In [16]:
consumidores_procesado = pipeline_procesado.fit_transform(consumidores)
consumidores_procesado.shape

(10000, 145)

Finalmente, necesitamos añadir un estimador al final para predecir en base a los datos transformados, pero primero se separa los datos en datos de entrenamiento y datos de test

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
     consumidores[variables_independientes], consumidores["CatRiesgo"],
     test_size=0.20, random_state=42
)

In [18]:
estimador = DecisionTreeClassifier()

Se debe realizar la separacion de datos en datos de entrenamiento y datos de test y ademas se debe establecer el estimador a usar en el modelo

In [19]:
pipeline_estimador = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', estimador)
])

In [20]:
pipeline_estimador.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'd_12_c_presenta_problema_expendio_sp_barrio_tipo',
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                                    'd_01_aporta_dinero_hogar_tipo',
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo', ...])]))]))])),
                ('estimador', DecisionTreeClassifier())])

In [21]:
predicciones = pipeline_estimador.predict(X_test)

In [22]:
predicciones

array([0, 2, 0, ..., 2, 1, 4], dtype=int64)

Se evalua el modelo segun lo obtenido

El beneficio de los pipelines, no solo es tener codigo mas legible y poder gestionar de forma ordenada todo el ciclo de vida del modelado, sino que los pipelines tienen todos los beneficios de los objetos de scikitlearn, por ejemplo, podemos usar validacion cruzada directamente con el pipeline.

In [23]:
resultados = {}

def exactitud_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.accuracy_score(y, preds)

def precision_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.precision_score(y, preds, average="weighted")

def exhaustividad_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.recall_score(y, preds, average="weighted")

def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.f1_score(y, preds, average="weighted")

In [24]:
def evaluar_modelo(prmPipeline,X, y, semillas):
    restultados_estimador = {}
    restultados_estimador["exactitud"] = cross_validate(prmPipeline, X, y,
                                           scoring=exactitud_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["precision"] = cross_validate(prmPipeline, X, y,
                                           scoring=precision_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["exhaustividad"] = cross_validate(prmPipeline, X, y,
                                           scoring=exhaustividad_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["f1_score"] = cross_validate(prmPipeline, X, y,
                                           scoring=f1_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    return restultados_estimador

In [25]:
#Funcion para poder visuallizar los resultados de las metricas
def ver_resultados():
    df_res = pd.DataFrame()
    for modelo, metricas in resultados.items():
        for metrica, valores in metricas.items():
            for medida, arreglo in valores.items():
                # Calcula la media de cada array y agrega una nueva fila al DataFrame
                df_res.loc[f"{modelo}_{metrica}", medida] = np.mean(arreglo)
                
    return df_res

In [26]:
estimador_agregado = pipeline_estimador.named_steps['estimador']
resultados["DT"] = evaluar_modelo(estimador_agregado, consumidores_procesado, consumidores.CatRiesgo, 50)

In [27]:
ver_resultados()

,fit_time,score_time,test_score,train_score
DT_exactitud,0.330438,0.003081,0.924300,1.0
DT_precision,0.328982,0.005497,0.924034,1.0
DT_exhaustividad,0.337132,0.007004,0.923900,1.0
DT_f1_score,0.351614,0.006541,0.921167,1.0


### Optimizacion de hiperparametros

En esta sección se configuran los diferentes parámetros del modelo, con el objetivo de mejorar los resultados de las métricas.

In [28]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform, loguniform

In [29]:
param_dist_random = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 10, 20, 30],
    'min_samples_leaf': [1, 5, 10, 20],
    'max_features': [None, 'sqrt', 'log2']
}

In [30]:
busqueda_random = RandomizedSearchCV(estimator=estimador, 
                   param_distributions=param_dist_random,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [31]:
pipeline_estimador_op = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random)
])

In [32]:
pipeline_estimador_op.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(estimator=DecisionTreeClassifier(),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'max_depth': [None, 10,
                                                                       20, 30,
                                                                       40, 50],
                                                         'max_features': [None,
                                                                          'sqrt',
                                                                          'log2'],
                                                         'min_samples_leaf': [1,
                                                                              5,
                                                                              10,
                                                                              20],
                                                         'min_samples_split': [2,
                                                                               10,
                                                                               20,
                                                                               30]},
                                    scoring='accuracy'))])

In [33]:
pipeline_estimador_op.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [34]:
print(busqueda_random.best_score_)
print(busqueda_random.best_estimator_)

0.927125
DecisionTreeClassifier(max_depth=20, min_samples_leaf=5)


In [35]:
pipeline_estimador_op_best = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random.best_estimator_)
])

In [36]:
pipeline_estimador_op_best.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'd_12_c_presenta_problema_expendio_sp_barrio_tipo',
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                                    'd_01_aporta_dinero_hogar_tipo',
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo', ...])]))]))])),
                ('estimador',
                 DecisionTreeClassifier(max_depth=20, min_samples_leaf=5))])

In [37]:
pipeline_estimador_op_best.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [38]:
estimador_best = pipeline_estimador_op_best.named_steps['estimador']
resultados["DT_best"] = evaluar_modelo(estimador_best, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
DT_exactitud,0.330438,0.003081,0.924300,1.000000
DT_precision,0.328982,0.005497,0.924034,1.000000
DT_exhaustividad,0.337132,0.007004,0.923900,1.000000
DT_f1_score,0.351614,0.006541,0.921167,1.000000
DT_best_exactitud,0.303011,0.002723,0.931300,0.965869
DT_best_precision,0.330596,0.006772,0.931957,0.965845
DT_best_exhaustividad,0.312723,0.005240,0.932500,0.965853
DT_best_f1_score,0.346022,0.006093,0.930955,0.965843


### Segunda optimizacion

In [39]:
param_dist_random2 = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 10, 20, 30],
    'min_samples_leaf': [1, 5, 10, 20]
}

In [40]:
busqueda_random2 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random2,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [41]:
pipeline_estimador_op2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random2)
])

In [42]:
pipeline_estimador_op2.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(estimator=DecisionTreeClassifier(),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'criterion': ['gini',
                                                                       'entropy'],
                                                         'max_depth': [None, 10,
                                                                       20, 30,
                                                                       40, 50],
                                                         'min_samples_leaf': [1,
                                                                              5,
                                                                              10,
                                                                              20],
                                                         'min_samples_split': [2,
                                                                               10,
                                                                               20,
                                                                               30],
                                                         'splitter': ['best',
                                                                      'random']},
                                    scoring='accuracy'))])

In [43]:
pipeline_estimador_op2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [44]:
print(busqueda_random2.best_score_)
print(busqueda_random2.best_estimator_)

0.93075
DecisionTreeClassifier(criterion='entropy', min_samples_leaf=5,
                       min_samples_split=10)


In [45]:
pipeline_estimador_op_best2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random2.best_estimator_)
])

In [46]:
pipeline_estimador_op_best2.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'd_12_c_presenta_problema_expendio_sp_barrio_tipo',
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                                    'd_01_aporta_dinero_hogar_tipo',
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo', ...])]))]))])),
                ('estimador',
                 DecisionTreeClassifier(criterion='entropy', min_samples_leaf=5,
                                        min_samples_split=10))])

In [47]:
pipeline_estimador_op_best2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [48]:
estimador_best2 = pipeline_estimador_op_best2.named_steps['estimador']
resultados["DT_best_2"] = evaluar_modelo(estimador_best2, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
DT_exactitud,0.330438,0.003081,0.924300,1.000000
DT_precision,0.328982,0.005497,0.924034,1.000000
DT_exhaustividad,0.337132,0.007004,0.923900,1.000000
DT_f1_score,0.351614,0.006541,0.921167,1.000000
DT_best_exactitud,0.303011,0.002723,0.931300,0.965869
DT_best_precision,0.330596,0.006772,0.931957,0.965845
DT_best_exhaustividad,0.312723,0.005240,0.932500,0.965853
DT_best_f1_score,0.346022,0.006093,0.930955,0.965843
DT_best_2_exactitud,0.301880,0.002997,0.932200,0.970353
DT_best_2_precision,0.310108,0.005159,0.933025,0.970388


### Tercera optimizacion

In [49]:
param_dist_random3 = {
    'criterion': ['entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 10, 20, 30],
    'min_samples_leaf': [1, 5, 10, 20],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
    'max_features': [None, 'sqrt', 'log2']
}

In [50]:
busqueda_random3 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random3,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [51]:
pipeline_estimador_op3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random3)
])

In [52]:
pipeline_estimador_op3.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                ('estimador',
                 RandomizedSearchCV(estimator=DecisionTreeClassifier(),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'criterion': ['entropy'],
                                                         'max_depth': [None, 10,
                                                                       20, 30,
                                                                       40, 50],
                                                         'max_features': [None,
                                                                          'sqrt',
                                                                          'log2'],
                                                         'min_samples_leaf': [1,
                                                                              5,
                                                                              10,
                                                                              20],
                                                         'min_samples_split': [2,
                                                                               10,
                                                                               20,
                                                                               30],
                                                         'min_weight_fraction_leaf': [0.0,
                                                                                      0.1,
                                                                                      0.2],
                                                         'splitter': ['best',
                                                                      'random']},
                                    scoring='accuracy'))])

In [53]:
pipeline_estimador_op3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [54]:
print(busqueda_random3.best_score_)
print(busqueda_random3.best_estimator_)

0.928375
DecisionTreeClassifier(criterion='entropy', max_depth=40, min_samples_leaf=5,
                       splitter='random')


In [55]:
pipeline_estimador_op_best3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random3.best_estimator_)
])

In [56]:
pipeline_estimador_op_best3.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                                    'd_01_aporta_dinero_hogar_tipo',
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo', ...])]))]))])),
                ('estimador',
                 DecisionTreeClassifier(criterion='entropy', max_depth=40,
                                        min_samples_leaf=5,
                                        splitter='random'))])

In [57]:
pipeline_estimador_op_best3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [58]:
estimador_best3 = pipeline_estimador_op_best3.named_steps['estimador']
resultados["DT_best_3"] = evaluar_modelo(estimador_best3, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
DT_exactitud,0.330438,0.003081,0.924300,1.000000
DT_precision,0.328982,0.005497,0.924034,1.000000
DT_exhaustividad,0.337132,0.007004,0.923900,1.000000
DT_f1_score,0.351614,0.006541,0.921167,1.000000
DT_best_exactitud,0.303011,0.002723,0.931300,0.965869
DT_best_precision,0.330596,0.006772,0.931957,0.965845
DT_best_exhaustividad,0.312723,0.005240,0.932500,0.965853
DT_best_f1_score,0.346022,0.006093,0.930955,0.965843
DT_best_2_exactitud,0.301880,0.002997,0.932200,0.970353
DT_best_2_precision,0.310108,0.005159,0.933025,0.970388


### Cuarta optimizacion

In [59]:
param_dist_random4 = {
    'criterion': ['entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 10, 20, 30],
    'min_samples_leaf': [1, 5, 10, 20],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
    'max_features': [None, 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 20, 30],
    'min_impurity_decrease': [0.0, 0.01, 0.1]
}

In [60]:
busqueda_random4 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random4,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [61]:
pipeline_estimador_op4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random4)
])

In [62]:
pipeline_estimador_op4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'criterion': ['entropy'],
                                                         'max_depth': [None, 10,
                                                                       20, 30,
                                                                       40, 50],
                                                         'max_features': [None,
                                                                          'sqrt',
                                                                          'log2'],
                                                         'max_leaf_nodes': [None,
                                                                            10,
                                                                            20,
                                                                            30],
                                                         'min_impurity_decrease': [0.0,
                                                                                   0.01,
                                                                                   0.1],
                                                         'min_samples_leaf': [1,
                                                                              5,
                                                                              10,
                                                                              20],
                                                         'min_samples_split': [2,
                                                                               10,
                                                                               20,
                                                                               30],
                                                         'min_weight_fraction_leaf': [0.0,
                                                                                      0.1,
                                                                                      0.2],
                                                         'splitter': ['best',
                                                                      'random']},
                                    scoring='accuracy'))])

In [63]:
pipeline_estimador_op4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [64]:
print(busqueda_random4.best_score_)
print(busqueda_random4.best_estimator_)

0.90425
DecisionTreeClassifier(criterion='entropy', max_depth=30, max_leaf_nodes=20,
                       min_impurity_decrease=0.01, min_samples_split=30)


In [65]:
pipeline_estimador_op_best4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random4.best_estimator_)
])

In [66]:
pipeline_estimador_op_best4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                                    'd_01_aporta_dinero_hogar_tipo',
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo', ...])]))]))])),
                ('estimador',
                 DecisionTreeClassifier(criterion='entropy', max_depth=30,
                                        max_leaf_nodes=20,
                                        min_impurity_decrease=0.01,
                                        min_samples_split=30))])

In [67]:
pipeline_estimador_op_best4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [68]:
estimador_best4 = pipeline_estimador_op_best4.named_steps['estimador']
resultados["DT_best_4"] = evaluar_modelo(estimador_best4, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
DT_exactitud,0.330438,0.003081,0.924300,1.000000
DT_precision,0.328982,0.005497,0.924034,1.000000
DT_exhaustividad,0.337132,0.007004,0.923900,1.000000
DT_f1_score,0.351614,0.006541,0.921167,1.000000
DT_best_exactitud,0.303011,0.002723,0.931300,0.965869
DT_best_precision,0.330596,0.006772,0.931957,0.965845
DT_best_exhaustividad,0.312723,0.005240,0.932500,0.965853
DT_best_f1_score,0.346022,0.006093,0.930955,0.965843
DT_best_2_exactitud,0.301880,0.002997,0.932200,0.970353
DT_best_2_precision,0.310108,0.005159,0.933025,0.970388


### Quinta optimizacion

In [69]:
param_dist_random5 = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 10, 20, 30],
    'min_samples_leaf': [1, 5, 10, 20],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
    'max_features': [None, 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 20, 30],
    'min_impurity_decrease': [0.0, 0.01, 0.1],
    'class_weight': [None, 'balanced'],
    'ccp_alpha': [0.0, 0.01, 0.02, 0.05, 0.1]
}

In [70]:
busqueda_random5 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random5,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [71]:
pipeline_estimador_op5 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random5)
])

In [72]:
pipeline_estimador_op5.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                         'class_weight': [None,
                                                                          'balanced'],
                                                         'criterion': ['gini',
                                                                       'entropy'],
                                                         'max_depth': [None, 10,
                                                                       20, 30,
                                                                       40, 50],
                                                         'max_features': [None,
                                                                          'sqrt',
                                                                          'log2'],
                                                         'max_leaf_nodes': [None,
                                                                            10,
                                                                            20,
                                                                            30],
                                                         'min_impurity_decrease': [0.0,
                                                                                   0.01,
                                                                                   0.1],
                                                         'min_samples_leaf': [1,
                                                                              5,
                                                                              10,
                                                                              20],
                                                         'min_samples_split': [2,
                                                                               10,
                                                                               20,
                                                                               30],
                                                         'min_weight_fraction_leaf': [0.0,
                                                                                      0.1,
                                                                                      0.2],
                                                         'splitter': ['best',
                                                                      'random']},
                                    scoring='accuracy'))])

In [73]:
pipeline_estimador_op5.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [74]:
print(busqueda_random5.best_score_)
print(busqueda_random5.best_estimator_)

0.9171249999999999
DecisionTreeClassifier(max_depth=40, max_leaf_nodes=20, min_samples_leaf=5)


In [75]:
pipeline_estimador_op_best5 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random5.best_estimator_)
])

In [76]:
pipeline_estimador_op_best5.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'd_12_c_presenta_problema_expendio_sp_barrio_tipo',
                                                                                                    'g_07_alguien_ofrecio_comprar_probar_sp_tipo',
                                                                                                    'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                                    'd_01_aporta_dinero_hogar_tipo',
                                                                                                    'g_02_a_num_amigos_consumen_sp_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo', ...])]))]))])),
                ('estimador',
                 DecisionTreeClassifier(max_depth=40, max_leaf_nodes=20,
                                        min_samples_leaf=5))])

In [77]:
pipeline_estimador_op_best5.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [78]:
estimador_best5 = pipeline_estimador_op_best5.named_steps['estimador']
resultados["DT_best_5"] = evaluar_modelo(estimador_best5, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
DT_exactitud,0.330438,0.003081,0.924300,1.000000
DT_precision,0.328982,0.005497,0.924034,1.000000
DT_exhaustividad,0.337132,0.007004,0.923900,1.000000
DT_f1_score,0.351614,0.006541,0.921167,1.000000
DT_best_exactitud,0.303011,0.002723,0.931300,0.965869
DT_best_precision,0.330596,0.006772,0.931957,0.965845
DT_best_exhaustividad,0.312723,0.005240,0.932500,0.965853
DT_best_f1_score,0.346022,0.006093,0.930955,0.965843
DT_best_2_exactitud,0.301880,0.002997,0.932200,0.970353
DT_best_2_precision,0.310108,0.005159,0.933025,0.970388


### Exportar con joblib el mejor pipeline obtenido

Tenemos que exportar 3 cosas para poder recrear el Pipeline:

**1. El pipeline entrenado.**

Podemos usar la libreria de scikit-learn `joblib` para guardar nuestro pipeline al disco duro. Joblib es una libreria basada en pickle que sirve para guardar objetos de python al disco duro, pero que está optimizado para scikit-learn.

In [79]:
import joblib

In [80]:
joblib.dump(pipeline_estimador_op_best3, 'pipeline_DT_best.pkl')

['pipeline_DT_best.pkl']

In [81]:
clf = joblib.load('C:/Users/Danny/Desktop/Analisis_Modelo/Alisis Danny/4 - Pipelines/Modelo DT/pipeline_DT_best.pkl')

In [82]:
clf.predict(consumidores.head())

array([0, 0, 0, 0, 1], dtype=int64)

**2. Los nombres de las columnas**

Necesitamos saber las columnas que espera el pipeline para poder saber cómo tenemos que formatear los datos en el futuro de forma que los reconozca.

In [83]:
import json
with open('columnas_consumidores.json', 'w') as fname:
    consumidores_columnas = consumidores.columns.tolist()
    json.dump(consumidores_columnas, fname)

**3. Los tipos de datos de cada variable**

Necesitamos que las variables de las nuevas observaciones estén en el mismo formato que espera el pipeline.

In [84]:
consumidores_dtypes = consumidores.dtypes
consumidores_dtypes = {col: consumidores[col].dtype for col in consumidores.columns}
joblib.dump(consumidores_dtypes, 'dtypes_consumidores.pkl')

['dtypes_consumidores.pkl']